<a href="https://colab.research.google.com/github/Ricardo50-dev/GA_FTIR_complex_network/blob/main/AG_FTIR_Redes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importando bibliotecas**

In [ ]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import igraph as ig
import rampy as rp
from copy import copy
import matplotlib.pyplot as plt
import random
from random import randrange
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import minmax_scale, normalize
from sklearn.metrics import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import pairwise, confusion_matrix
from scipy.signal import savgol_filter
from scipy.signal import gauss_spline
from sklearn.neighbors import KDTree
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install igraph
!pip install rampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.4 MB/s eta 0:00:00


**População**

In [ ]:
def inicia_populacao(Tpop, wavs):
  Pop = np.zeros((Tpop,wavs.size))
  for i in range(Tpop):
    Pop[i] = np.random.randint(2, size=wavs.size)
  return Pop

**Leitura dos dados**

In [ ]:
def load_ftir_data(filename): #Leitura do arquivo FTIR
    data = np.loadtxt(filename)
    return data[:,:-1], data[:,-1].astype(int)

#def main(): Iniciando o programa.
wavsname = "wavenumbers.dat"
filename = "dataset_cancboca_bruto.dat"

#load input data
X, y = load_ftir_data(filename)
y = np.where(y <= -1, 0, y)
#load wavs size
wavs = np.loadtxt(wavsname)

**Parâmetros do AG**

In [ ]:
Tpop = 10
Taxa_mut = 0.2
Geracoes = 10
T_cross = 0.4
k = 7
med_rede = "clustcoefficient"
#modelo pode ser SVM, RF ou LDA

**Inicia população**

In [ ]:
Pop = inicia_populacao(Tpop, wavs)

In [ ]:
np.shape(Pop)

(10, 1867)

**Mutação, Crossover e Torneio**

In [ ]:
def Mutacao(pop, Taxa_mut):
  for i in range(pop.shape[0]):
    sorteio = np.random.randint(pop.shape[1]-1, size=1) # Sorteia uma posição do array
    p = random.random() #Probabiliade da mutação

    if p < Taxa_mut :
      if pop[i][sorteio] == 1:
        pop[i][sorteio] = 0
      else:
        pop[i][sorteio] = 1

  return pop

In [ ]:
def selecao_torneio(pop, k, Tpop):
  participantes = np.random.randint(Tpop, size=k) #Sorteia k individuos para o torneio
  best = 0
  for i in range (k):
    fit = pop[participantes[i]][-1]
    if fit > best:
      best = fit
      ganhador = pop[participantes[i]]

  return ganhador

In [ ]:
def crossover(pai1, pai2):
  x = np.random.randint(max(pai1.size,pai2.size), size=1) # Sorteia uma posição do array
  x = int(x)
  tmp = pai2[:x].copy()
  pai2[:x], pai1[:x]  = pai1[:x], tmp

  filhos = np.vstack([pai1,pai2])

  return filhos

**Gerador da rede, Medidas de rede e Funções complementares**

In [ ]:
def graphgen(k, X, y, med_rede):

    # pega o num de classes
    numclasses = len(np.unique(y))

    # X = gauss_spline(X, len(X))

    euclidean_dist = pairwise.cosine_similarity(X)

    # preenche a diagonal principal com valor infinito
    np.fill_diagonal(euclidean_dist, np.inf)

    # ordena as distancias por objeto em ordem ascendente
    #print(euclidean_dist)
    ind_ranking = np.argsort(euclidean_dist, axis=1)[:, :k]

    # inicializa uma mascara com False, num de objetos x k
    mask = np.zeros((len(X), k)).astype(int)

    # preenche os campos True da mascara, vizinho mais proximos que sao da mesma clase
    lista_grafos = []
    for i in range(len(ind_ranking)):
        mask[i] = (y[ind_ranking[i]] == y[i])

    # captura os indices True da mascara, linha e coluna
    links = mask.nonzero()

    # atribui os indices das linhas
    sources = links[0]

    # atribui o indice real dos objetos das colunas
    targets = ind_ranking[links]

    # inicializa funcao para mapeamento dos vertices
    map_vertices = np.zeros(len(ind_ranking)).astype(int) - 1

    # inicializa variavel para guardar o calculo das medidas
    measures = np.zeros((numclasses, 1))
    for l in range(numclasses):

        # captura todos os objetos em sources que pertencem a classe l
        lsources = np.where(y[sources] == l)[0]

        # captura todos os objetos da base que pertencem a classe l
        all_vertices = np.where(y == l)[0]

        # recebe todos os objetos da classe l que estao conectados
        unique_vertices = np.unique(
            np.append(sources[lsources], targets[lsources])).astype(int)

        # recebe os demais objetos da classe l que nao estao conectados
        unique_vertices = np.unique(
            np.append(unique_vertices, all_vertices)).astype(int)

        # popula a funcao de mapeamento dos vertices com os objetos da classe l
        map_vertices[unique_vertices] = np.arange(
            len(unique_vertices)).astype(int)

        # cria o grafo para classe l
        subg = ig.Graph(len(unique_vertices), list(zip(map_vertices[sources[lsources]].astype(
            int), map_vertices[targets[lsources]].astype(int))))

        # popula a lista de grafos
        lista_grafos.append(subg)

        # calcula as medidas de rede associadas ao grafo l
        if med_rede == 'assortativity':
            measures[l, 0] = Assortativity(subg)
        elif med_rede == 'clustcoefficient':
            measures[l, 0] = ClustCoefficient(subg)
        elif med_rede == 'avgdegree':
            measures[l, 0] = AvgDegree(subg)
        elif med_rede == 'betweenness':
            measures[l, 0] = Betweenness(subg)
        elif med_rede == 'avgpathlength':
            measures[l, 0] = AvgPathLength(subg)
        elif med_rede == 'closeness':
            measures[l, 0] = Closeness(subg)
        '''measures[l, 0] = Assortativity(subg)
        measures[l, 1] = ClustCoefficient(subg)
        measures[l, 2] = AvgDegree(subg)
        measures[l, 3] = Betweenness(subg)
        measures[l, 4] = AvgPathLength(subg)
        measures[l, 5] = Closeness(subg)'''

    return lista_grafos, map_vertices, measures

In [ ]:
def Assortativity(graph):
    return ig.Graph.assortativity_degree(graph)


def ClustCoefficient(graph):
    return ig.Graph.transitivity_avglocal_undirected(graph)


def AvgDegree(graph):
    return np.mean(ig.Graph.degree(graph))


def Betweenness(graph):
    return np.mean(ig.Graph.betweenness(graph))


def AvgPathLength(graph):
    return ig.Graph.average_path_length(graph)


def Closeness(graph):
    return np.mean(ig.Graph.closeness(graph))

In [ ]:
def deep_copy_graphs(lista_grafos):
    newlist = []
    for i in range(len(lista_grafos)):
        newlist.append(ig.Graph.copy(lista_grafos[i]))

    return newlist

def sample_smoothing_differentiation(X):
    return savgol_filter(X, 25, 4, 2, axis=1)


def sample_normalization(X):
    return X/(np.linalg.norm(X,2,axis=1).reshape(-1,1))


def sample_amida1_normalization(X, amida1_ids):
    return X/(np.max(X[:,amida1_ids],axis=1).reshape(-1,1))

**Função Fitness**

fit_rede = gera um dicionario contento valores de ACC, SEN, ESP, STD, MD

In [ ]:
def fit_rede(X, y, med_rede, k):
    NN1 = True
    acc1NN = []

    # matriz para os resultados, 30 analises, 15 valor de k, e 6 medidas
    acc_results = np.zeros((30, 17, 1))
    sen_results = np.zeros((30, 17, 1))
    esp_results = np.zeros((30, 17, 1))

    # rodar para cada valor de k de 1 a 15
    #for k in range(1, 16):
    k = 13
    # seta um valor seed para gerar os numeros pseudo-aleatorios
    np.random.seed(1000)

    # KNN para construcao da base com k = 1
    knn = KNeighborsClassifier(n_neighbors=1)

    # validacao cruzada, a base eh dividida em 10 partes e cada uma eh executada 3 vezes como treino
    rskf = RepeatedStratifiedKFold(
        n_splits=10, n_repeats=3, random_state=10)

    # Separa os dados de treino e teste utilizando validacao cruzada
    for b1, (train_index, test_index) in enumerate(rskf.split(X, y)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # conta quantas vezes cada valor de y apareceu
        # 1NN - baseline
        if NN1:
            # quais sao as classes para serem classificada?
            knn.fit(X_train, y_train)

            # insere o resultado do knn em uma lista
            acc1NN.append(knn.score(X_test, y_test))
            # print("1: ", acc)

        # Network measures
        # Define a matriz predicted com o tamanho do teste, a classe, e o numero de medidas
        predicted = np.ones((len(X_test), len(np.unique(y)), 1)) * np.inf

        # manda para a funcao graphgen, o valor de k, e
        lista_grafos, map_vertices, measures = graphgen(
            k, X_train, y_train, med_rede)
        # calcula a distancia euclidiana entre os pares de teste e o objeto
        tree = KDTree(X_train)
        dist, ind = tree.query(X_test, k=3)

        for i in range(len(X_test)):

            # ordena as distancias por objeto em ordem ascendente
            ids = ind[i - 1]
            #ids = np.argsort(dist[i])[:k]

            # newgraph recebe uma copia de lista_grafos
            newgraph = deep_copy_graphs(lista_grafos)

            # j eh o indice do objeto de treino e c a classe dele
            for j, c in enumerate(y_train[ids]):
                # cria um id para o novo vertice no grafo da classe c
                newid = lista_grafos[c].vcount()
                # adiciona o vertice do grafo para o novo objeto
                newgraph[c].add_vertex(newid)
                # verifica se o vertice em que o novo objeto sera conectado ja esta no grafo
                if map_vertices[ids[j]] > newid:
                    newgraph[c].add_vertex(map_vertices[ids[j]])

                newgraph[c].add_edge(newid, map_vertices[ids[j]])

            # para cada uma das classes
            for c in np.unique(y_train[ids]):
                subg = newgraph[c]
                # calcula as medidas
                if med_rede == 'assortativity':
                    predicted[i, c, 0] = abs(Assortativity(subg) - measures[c, 0])
                elif med_rede == 'clustcoefficient':
                    predicted[i, c, 0] = abs(ClustCoefficient(subg) - measures[c, 0])
                elif med_rede == 'avgdegree':
                    predicted[i, c, 0] = abs(AvgDegree(subg) - measures[c, 0])
                elif med_rede == 'betweenness':
                    predicted[i, c, 0] = abs(Betweenness(subg) - measures[c, 0])
                elif med_rede == 'avgpathlength':
                    predicted[i, c, 0] = abs(
                        AvgPathLength(subg) - measures[c, 0])
                elif med_rede == 'closeness':
                    predicted[i, c, 0] = abs(Closeness(subg) - measures[c, 0])

        # calcula a acuracia, sensibilidade e especificidade
        Matriz = np.zeros((len(y_test),2))
        Matriz[:,:-1] = np.tile(y_test, (1, 1)).T
        Matriz[:,-1:] = np.argmin(predicted, axis=1)
        matriz_conf = np.zeros((2,2))
        tp = tn = fp = fn = 0
        for p in range(len(y_test)):
            for l in range(2):
                if l == 0:
                    ini = Matriz[p][l]
                else:
                    fim = Matriz[p][l]
            if ini == fim == 1:
                tp += 1
            elif ini == fim == 0:
                tn += 1
            elif ini == 0 and fim == 1:
                fp += 1
            elif ini == 1 and fim == 0:
                fn += 1
        matriz_conf[0][0] = tp
        matriz_conf[0][1] = fn
        matriz_conf[1][0] = fp
        matriz_conf[1][1] = tn
        sensibilidade = tp/(tp + fn)
        especificidade = tn/(fp + tn)
        sen_results[b1, k - 1] = sensibilidade
        esp_results[b1, k - 1] = especificidade
        acc_results[b1, k - 1] = np.mean(np.tile(y_test, (1, 1)).T ==
                                        np.argmin(predicted, axis=1), axis=0)

    #Calcula todas a medidas avaliativas na mão
    fit = np.max(np.mean(acc_results, axis=0))
    std = np.max(np.std(acc_results, axis=0))
    sen = np.max(np.mean(sen_results, axis=0))
    esp = np.max(np.mean(esp_results, axis=0))
    md = (sen + esp) / 2
    #Retorna o dicionario do individuo organizadamente entre chave e valor
    dicionarioIndividuo = dict(acc=fit, std=std, sen=sen, esp=esp, md=md)

    return dicionarioIndividuo

In [ ]:
def fitness_pop(pop, x, y, med_rede, k):
  fit = np.zeros([pop[:,1].size, 1],dtype=float) #Criacao da array do fit do tamanho da populacao, a matriz contem 1 coluna com pop.size linhas.
  for i in range(pop.shape[0]):
    ind = pop[i]
    data = x
    amida1_ids = np.where(np.logical_and(wavs >= 1630, wavs <= 1660))[0]
    data = sample_amida1_normalization(data, amida1_ids)
    data = data[:,ind[0:].nonzero()[0]]
    resultado = fit_rede(data, y, med_rede, k)
    fit[i,0] = resultado['acc']

  return fit

In [ ]:
fit = fitness_pop(Pop, X, y, med_rede, k)
Pop = np.hstack([Pop,fit])

In [ ]:
Pop

array([[1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.61587302],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.62063492],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.63809524],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.58809524],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.59126984],
       [1.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.6031746 ]])

**Iniciando o AG**

In [ ]:
lim = 0
T_cross = T_cross*Tpop
#Taxa de crossover define o tanto de filhos novos, nesse caso 0.4 * 100 = 40 filhos novos
while (lim <= Geracoes):
  lim += 1

  Qtd_f = int(T_cross/2)

  filhos = np.zeros((int(Qtd_f*2), wavs.size))
  filho1 = np.zeros((int(Qtd_f), wavs.size))
  filho2 = np.zeros((int(Qtd_f), wavs.size))

  for i in range(int(Qtd_f)):
    selecao_torneio(Pop, 4, Tpop)
    pai1 = selecao_torneio(Pop, 4, Tpop)
    pai2 = selecao_torneio(Pop, 4, Tpop) #Crossover com tamanho T_cross, definido como 40% da população dos pais da populacao
    filho1[i],filho2[i] = crossover(pai1[:-1], pai2[:-1])

  filhos = np.vstack([filho1,filho2])
  filhos = Mutacao(filhos, Taxa_mut)

  fit_filhos = fitness_pop(filhos, X, y, med_rede, k)
  filhos = np.hstack([filhos,fit_filhos])

  Nova_pop = np.vstack([Pop,filhos]) # Reinsercao
  Nova_pop_sorted = Nova_pop[:,-1].argsort()
  Nova_pop_sorted = Nova_pop_sorted[::-1]
  Pop = Nova_pop[Nova_pop_sorted][0:Tpop, :] # Atualiza pop pegando os N melhores individuos

  print("Geração: ", lim, "Melhor fitness: ", max(Pop[:,-1]))

Geração:  1 Melhor fitness:  0.6380952380952378
Mutacao
Geração:  2 Melhor fitness:  0.6380952380952378
Mutacao
Geração:  3 Melhor fitness:  0.6380952380952378
Mutacao
Geração:  4 Melhor fitness:  0.6380952380952378
Mutacao
Geração:  5 Melhor fitness:  0.6380952380952378
Mutacao
Geração:  6 Melhor fitness:  0.6380952380952378
Geração:  7 Melhor fitness:  0.6380952380952378
Geração:  8 Melhor fitness:  0.6380952380952378
Geração:  9 Melhor fitness:  0.6380952380952378
Geração:  10 Melhor fitness:  0.6380952380952378
Mutacao
Geração:  11 Melhor fitness:  0.6380952380952378
